In [2]:
pip install xgboost

    100% |████████████████████████████████| 127.6MB 59kB/s eta 0:00:012
Note: you may need to restart the kernel to use updated packages.


In [123]:
from datetime import datetime
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np 
import pandas as pd
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.decomposition import PCA

# Считываем данные

In [124]:
bike_dataframe = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/bike_sharing/train.csv")
bike_dataframe.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [125]:
test_dataframe = pd.read_csv("/home/emin/PycharmProjects/machine_learning/data/bike_sharing/test.csv")
test_dataframe.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [126]:
sample_submission = pd.read_csv('/home/emin/PycharmProjects/machine_learning/data/bike_sharing/sampleSubmission.csv')
sample_submission.head()

,datetime,count
0,2011-01-20 00:00:00,0
1,2011-01-20 01:00:00,0
2,2011-01-20 02:00:00,0
3,2011-01-20 03:00:00,0
4,2011-01-20 04:00:00,0


# Создадим dataframe и объединим в него bike_dataframe, test_dataframe .

In [127]:
Y_bike = bike_dataframe['count']
names = list(test_dataframe)
bike_dataframe = bike_dataframe[names]
dataframe = pd.concat((bike_dataframe, test_dataframe))
dataframe.isnull().sum()

datetime      0
season        0
holiday       0
workingday    0
weather       0
temp          0
atemp         0
humidity      0
windspeed     0
dtype: int64

# Преобразуем datatime в объект даты-времени и  формируем как отдельные столбцы(год, месяц, день, час, день недели(понедельник, вторник и т.д)). После удаляем столбец datatime

In [128]:
dataframe['datetime'] = pd.to_datetime(dataframe['datetime'])
dataframe['year'] = dataframe['datetime'].dt.year
dataframe['month'] = dataframe['datetime'].dt.month
dataframe['day'] = dataframe['datetime'].dt.day
dataframe['hour'] = dataframe['datetime'].dt.hour
dataframe['dayofweek'] = dataframe['datetime'].dt.dayofweek
dataframe = dataframe.drop(columns='datetime')

# Если дней в недели 5 или 6 обьявляем переменную weekend как True

In [129]:
dataframe = dataframe.drop(columns=['month', 'day'])
dataframe['weekend'] = (dataframe['dayofweek'] == 5) | (dataframe['dayofweek'] == 6)
dataframe.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,hour,dayofweek,weekend
0,1,0,0,1,9.84,14.395,81,0.0,2011,0,5,True
1,1,0,0,1,9.02,13.635,80,0.0,2011,1,5,True
2,1,0,0,1,9.02,13.635,80,0.0,2011,2,5,True
3,1,0,0,1,9.84,14.395,75,0.0,2011,3,5,True
4,1,0,0,1,9.84,14.395,75,0.0,2011,4,5,True


# Разобьем отдельно на 4 сезона(season__1,season2__2 и т.д) и удалим собственно столбец season, так как нам он уже не нужен.

In [130]:
season = pd.get_dummies(dataframe['season'], prefix='season_')
dataframe = pd.concat((dataframe, season), axis=1)
dataframe = dataframe.drop(columns='season')
dataframe.head()

,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,hour,dayofweek,weekend,season__1,season__2,season__3,season__4
0,0,0,1,9.84,14.395,81,0.0,2011,0,5,True,1,0,0,0
1,0,0,1,9.02,13.635,80,0.0,2011,1,5,True,1,0,0,0
2,0,0,1,9.02,13.635,80,0.0,2011,2,5,True,1,0,0,0
3,0,0,1,9.84,14.395,75,0.0,2011,3,5,True,1,0,0,0
4,0,0,1,9.84,14.395,75,0.0,2011,4,5,True,1,0,0,0


# Также разбиваем погоду на 4 столбца(weather__1, weather__2 и т.д) и удаляем столбец weather.

In [131]:
weather = pd.get_dummies(dataframe['weather'], prefix='weather_')
dataframe = pd.concat((dataframe, weather), axis=1)
dataframe = dataframe.drop(columns='weather')
dataframe.head()

,holiday,workingday,temp,atemp,humidity,windspeed,year,hour,dayofweek,weekend,season__1,season__2,season__3,season__4,weather__1,weather__2,weather__3,weather__4
0,0,0,9.84,14.395,81,0.0,2011,0,5,True,1,0,0,0,1,0,0,0
1,0,0,9.02,13.635,80,0.0,2011,1,5,True,1,0,0,0,1,0,0,0
2,0,0,9.02,13.635,80,0.0,2011,2,5,True,1,0,0,0,1,0,0,0
3,0,0,9.84,14.395,75,0.0,2011,3,5,True,1,0,0,0,1,0,0,0
4,0,0,9.84,14.395,75,0.0,2011,4,5,True,1,0,0,0,1,0,0,0


# Создадим pca для сокращения размерности данных(используем для температуры, чтобы избавиться от temp и atemp).

In [132]:
pca = PCA(n_components=1)
pca.fit(dataframe[['temp', 'atemp']])

print(f" Доля информации (доля от общей дисперсии): {pca.explained_variance_ratio_}")

 Доля информации (доля от общей дисперсии): [0.99388034]


In [133]:
dataframe['temperature'] = pca.fit_transform(dataframe[['temp','atemp']])
dataframe = dataframe.drop(columns=['temp', 'atemp'])
dataframe.head()

,holiday,workingday,humidity,windspeed,year,hour,dayofweek,weekend,season__1,season__2,season__3,season__4,weather__1,weather__2,weather__3,weather__4,temperature
0,0,0,81,0.0,2011,0,5,True,1,0,0,0,1,0,0,0,-14.045296
1,0,0,80,0.0,2011,1,5,True,1,0,0,0,1,0,0,0,-15.159684
2,0,0,80,0.0,2011,2,5,True,1,0,0,0,1,0,0,0,-15.159684
3,0,0,75,0.0,2011,3,5,True,1,0,0,0,1,0,0,0,-14.045296
4,0,0,75,0.0,2011,4,5,True,1,0,0,0,1,0,0,0,-14.045296


Если скорость ветра равна нулю, то присваеваем среднее значение скорости ветра.

In [134]:
dataframe.loc[dataframe['windspeed'] == 0, 'windspeed'] = dataframe['windspeed'].mean()
dataframe.head()
dataframe[0:-1]

,holiday,workingday,humidity,windspeed,year,hour,dayofweek,weekend,season__1,season__2,season__3,season__4,weather__1,weather__2,weather__3,weather__4,temperature
0,0,0,81,12.73654,2011,0,5,True,1,0,0,0,1,0,0,0,-14.045296
1,0,0,80,12.73654,2011,1,5,True,1,0,0,0,1,0,0,0,-15.159684
2,0,0,80,12.73654,2011,2,5,True,1,0,0,0,1,0,0,0,-15.159684
3,0,0,75,12.73654,2011,3,5,True,1,0,0,0,1,0,0,0,-14.045296
4,0,0,75,12.73654,2011,4,5,True,1,0,0,0,1,0,0,0,-14.045296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6487,0,1,48,8.99810,2012,18,0,False,1,0,0,0,0,1,0,0,-14.050731
6488,0,1,60,11.00140,2012,19,0,False,1,0,0,0,0,1,0,0,-14.606960
6489,0,1,60,11.00140,2012,20,0,False,1,0,0,0,0,1,0,0,-14.606960
6490,0,1,60,11.00140,2012,21,0,False,1,0,0,0,1,0,0,0,-14.606960


In [135]:
X_bike = dataframe[:len(bike_dataframe)]
X_test = dataframe[len(bike_dataframe):]
Y_bike = np.log(Y_bike + 1)
print(X_bike.shape, Y_bike.shape, X_test.shape)

(10886, 17) (10886,) (6493, 17)


In [136]:
xg_model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

# Обучаем модель и делаем прогнозы.

In [137]:
xg_model.fit(X_bike, Y_bike)
log_predictions = xg_model.predict(X_test)
prediction = np.exp(log_predictions) - 1
sample_submission['count'] = prediction
sample_submission.to_csv('/home/emin/PycharmProjects/machine_learning/data/bike_sharing/submission.csv', index=False)
sample_submission.head()

,datetime,count
0,2011-01-20 00:00:00,14.305049
1,2011-01-20 01:00:00,4.802293
2,2011-01-20 02:00:00,3.664081
3,2011-01-20 03:00:00,2.211867
4,2011-01-20 04:00:00,2.076866


# Результат: 0.38376